# Lib


In [ ]:
import math
import unittest
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.cm as cm


class Trail:
    def __init__(self, f, headers, cap, precision):
        self.cap = lambda x: cap(x, precision)
        self.f = f
        self.fcap = lambda x: self.cap(f(x))
        self.headers = headers
        self.precision = precision
        self.steps = []

    def record(self, *args):
        self.steps.append(args)

    def __getitem__(self, index):
        return self.steps[index]

    def __iter__(self):
        for step in self.steps:
            yield step

    def __len__(self):
        return len(self.steps)

    def __str__(self):
        return tabulate(
            self.steps,
            headers=self.headers,
            floatfmt=f".{self.precision}f",
            showindex=True,
        )

    @property
    def result(self):
        return self[-1][0]

    def plot(self):
        colors = cm.viridis(np.linspace(0, 1, len(self)))
        plt.subplots()

        (a, b) = self.bounds
        pad = (b - a) * 0.1
        x = np.arange(a - pad, b + pad, 0.01)

        plt.plot(x, self.f(x), label="f(x)")

        for i, color in zip(range(1, len(self)), colors):
            [(x1, y1), (x2, y2)] = self.points(i)

            if x1 != x2:
                slope = (y2 - y1) / (x2 - x1)
                intercept = y1 - slope * x1

                plt.plot(x, x * slope + intercept, label=i, color=color)
                plt.scatter([x1, x2], [y1, y2], color=color)

        plt.grid(True)
        plt.legend()


class RaphsonTrail(Trail):
    def __init__(self, f, fd, cap, precision):
        Trail.__init__(self, f, ["x", "f(x)", "f'(x)"], cap, precision)
        self.fd = fd
        self.fdcap = lambda x: cap(fd(x), precision)

    def record(self, x):
        self.steps.append([x, self.fcap(x), self.fdcap(x)])

    def points(self, i):
        x1, x2 = self[i - 1][0], self[i][0]
        y1, y2 = self.f(x1), 0
        return [(x1, y1), (x2, y2)]

    @property
    def bounds(self):
        return (self[-1][0], self[0][0])


class SecantTrail(Trail):
    def __init__(self, f, cap, precision):
        Trail.__init__(self, f, ["x", "f(x)"], cap, precision)

    def record(self, x):
        self.steps.append([x, self.fcap(x)])

    def points(self, i):
        x1, x2 = self[i - 1][0], self[i][0]
        y1, y2 = self[i - 1][1], self[i][1]
        return [(x1, y1), (x2, y2)]

    @property
    def bounds(self):
        points = [point[0] for point in self]
        return (min(points), max(points))


class TwoPointTrail(Trail):
    def __init__(self, f, cap, precision):
        Trail.__init__(self, f, ["a", "b", "x", "f(a)", "f(b)", "f(x)"], cap, precision)

    def record(self, a, b, x):
        self.steps.append([a, b, x, self.fcap(a), self.fcap(b), self.fcap(x)])

    @property
    def result(self):
        return self[-1][2]

    def points(self, i):
        x1, x2 = self[i][0], self[i][1]
        y1, y2 = self.f(x1), self.f(x2)
        return [(x1, y1), (x2, y2)]

    @property
    def bounds(self):
        return (self[0][0], self[0][1])


def trunc(x, digits):
    n = 10**digits
    return int(x * n) / float(n)


def find_root(f, a, b, update, tolerance=None, max_iterations=None, truncate=True, precision=4):
    cap = trunc if truncate == True else round
    trail = TwoPointTrail(f, cap, precision)

    i = 1
    while True:
        if f(a) * f(b) >= 0:
            raise Exception(f"f doesn't have a root between {a} and {b}")

        x = cap(update(a, b), precision)

        trail.record(a, b, x)

        if tolerance is not None and abs(f(x)) < tolerance:
            break

        if max_iterations is not None and i >= max_iterations:
            break

        if f(a) * f(x) >= 0:
            a = x
        else:
            b = x

        i += 1

    return trail

# Bissecção

https://numericalculator.canbula.com/bisection

In [ ]:
def bisection(f, a, b, tolerance=None, max_iterations=None, truncate=True, precision=4):
    return find_root(
        f=f,
        a=a,
        b=b,
        update=lambda a, b: (a + b) / 2,
        tolerance=tolerance,
        max_iterations=max_iterations,
        truncate=truncate,
        precision=precision,
    )

In [ ]:
class TestBisection(unittest.TestCase):
    def test_raises_when_no_root(self):
        with self.assertRaisesRegex(Exception, "f doesn't have a root between 1 and 2"):
            bisection(f=lambda x: x, a=1, b=2, tolerance=0.1)

    def test_first_exercise(self):
        trail = bisection(f=lambda x: x**3 - 9 * x + 5, a=0.5, b=1, tolerance=0.01)

        self.assertEqual(trail.result, 0.5781)

        #                      n    a       b       m       f(a)    f(b)     f(m)
        self.assertEqual(trail[0], [0.5000, 1.0000, 0.7500, 0.6250, -3.0000, -1.3281])
        self.assertEqual(trail[1], [0.5000, 0.7500, 0.6250, 0.6250, -1.3281, -0.3808])
        self.assertEqual(trail[2], [0.5000, 0.6250, 0.5625, 0.6250, -0.3808, 0.1154])
        self.assertEqual(trail[3], [0.5625, 0.6250, 0.5937, 0.1154, -0.3808, -0.1340])
        self.assertEqual(trail[4], [0.5625, 0.5937, 0.5781, 0.1154, -0.1340, -0.0096])

        trail.plot()

    def test_second_exercise(self):
        trail = bisection(f=lambda x: math.e**x - x - 2, a=-2, b=0, max_iterations=5)

        self.assertEqual(trail.result, -1.8125)

        #                      n     a        b        x       f(a)     f(b)     f(x)
        self.assertEqual(trail[0], [-2.0000, 0.0000, -1.0000, 0.1353, -1.0000, -0.6321])
        self.assertEqual(trail[1], [-2.0000, -1.0000, -1.5000, 0.1353, -0.6321, -0.2768])
        self.assertEqual(trail[2], [-2.0000, -1.5000, -1.7500, 0.1353, -0.2768, -0.0762])
        self.assertEqual(trail[3], [-2.0000, -1.7500, -1.8750, 0.1353, -0.0762, 0.0283])
        self.assertEqual(trail[4], [-1.8750, -1.7500, -1.8125, 0.0283, -0.0762, -0.0242])

        trail.plot()


unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestBisection))

# Falsa Posição

https://numericalculator.canbula.com/falseposition

In [ ]:
def false_posittion(f, a, b, tolerance=None, max_iterations=None, truncate=True, precision=4):
    return find_root(
        f=f,
        a=a,
        b=b,
        update=lambda a, b: (a * f(b) - b * f(a)) / (f(b) - f(a)),
        tolerance=tolerance,
        max_iterations=max_iterations,
        truncate=truncate,
        precision=precision,
    )

In [ ]:
import unittest


class TestFalsePosition(unittest.TestCase):
    def test_raises_when_no_root(self):
        with self.assertRaisesRegex(Exception, "doesn't have a root between 1 and 2"):
            false_posittion(f=lambda x: x, a=1, b=2, tolerance=0.1)

    def test_first_exercise(self):
        trail = false_posittion(
            f=lambda x: x**3 - 9 * x + 5,
            a=0,
            b=1,
            tolerance=0.0005,
        )

        self.assertEqual(trail.result, 0.5769)

        #                      n    a       b       x       f(a)     f(b)     f(x)
        self.assertEqual(trail[0], [0.0000, 1.0000, 0.6250, 5.0000, -3.0000, -0.3808])
        self.assertEqual(trail[1], [0.0000, 0.6250, 0.5807, 5.0000, -0.3808, -0.0304])
        self.assertEqual(trail[2], [0.0000, 0.5807, 0.5771, 5.0000, -0.0304, -0.0017])
        self.assertEqual(trail[3], [0.0000, 0.5771, 0.5769, 5.0000, -0.0017, 0.0000])

        trail.plot()

    def test_second_exercise(self):
        trail = false_posittion(
            f=lambda x: 2 * x**3 + 5 * x**2 - 8 * x - 10,
            a=0,
            b=3,
            tolerance=0.05,
            precision=6,
        )

        self.assertEqual(trail.result, 1.672066)

        self.assertEqual(trail[0], [0.000000, 3.000000, 0.400000, -10.000000, 65.000000, -12.272000])
        self.assertEqual(trail[1], [0.400000, 3.000000, 0.812920, -12.272000, 65.000000, -12.124747])
        self.assertEqual(trail[2], [0.812920, 3.000000, 1.156749, -12.124747, 65.000000, -9.468032])
        self.assertEqual(trail[3], [1.156749, 3.000000, 1.391104, -9.468032, 65.000000, -6.068933])
        self.assertEqual(trail[4], [1.391104, 3.000000, 1.528495, -6.068933, 65.000000, -3.404438])
        self.assertEqual(trail[5], [1.528495, 3.000000, 1.601730, -3.404438, 65.000000, -1.767543])
        self.assertEqual(trail[6], [1.601730, 3.000000, 1.638746, -1.767543, 65.000000, -0.880858])
        self.assertEqual(trail[7], [1.638746, 3.000000, 1.656946, -0.880858, 65.000000, -0.430026])
        self.assertEqual(trail[8], [1.656946, 3.000000, 1.665772, -0.430026, 65.000000, -0.207838])
        self.assertEqual(trail[9], [1.665772, 3.000000, 1.670024, -0.207838, 65.000000, -0.099963])
        self.assertEqual(trail[10], [1.670024, 3.000000, 1.672066, -0.099963, 65.000000, -0.047964])

        trail.plot()


unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestFalsePosition))

# Newton Raphson

https://numericalculator.canbula.com/newton

In [ ]:
def newton_raphson(f, fd, x, tolerance=None, max_iterations=None, truncate=True, precision=4):
    cap = trunc if truncate == True else round
    trail = RaphsonTrail(f, fd, cap, precision)

    i = 1
    while True:
        trail.record(x)

        if tolerance is not None and abs(f(x)) < tolerance:
            break

        if max_iterations is not None and i >= max_iterations:
            break

        x = cap(x - (f(x) / fd(x)), precision)

        i += 1

    return trail

In [ ]:
class TestNewtonRaphson(unittest.TestCase):
    def test_first_exercise(self):
        f = lambda x: x**3 - 9 * x + 3
        fd = lambda x: 3 * x**2 - 9

        trail = newton_raphson(f=f, fd=fd, x=0.75, tolerance=0.01, max_iterations=4, precision=3)

        self.assertEqual(trail[0], [0.750, -3.328, -7.312])
        self.assertEqual(trail[1], [0.294, 0.379, -8.740])
        self.assertEqual(trail[2], [0.337, 0.005, -8.659])
        self.assertEqual(trail.result, 0.337)

        trail.plot()

    def test_second_exercise(self):
        f = lambda x: 2 * x**3 + np.log(x) - 5
        fd = lambda x: 6 * x**2 + 1 / x

        trail = newton_raphson(
            f=f,
            fd=fd,
            x=2,
            tolerance=10e-7,
            max_iterations=6,
            truncate=False,
            precision=5,
        )

        self.assertEqual(trail[0], [2.00000, 11.69315, 24.50000])
        self.assertEqual(trail[1], [1.52273, 2.48203, 14.56896])
        self.assertEqual(trail[2], [1.35237, 0.24857, 11.71287])
        self.assertEqual(trail[3], [1.33115, 0.00353, 11.38299])
        self.assertEqual(trail[4], [1.33084, 0.00001, 11.37822])
        self.assertEqual(trail[5], [1.33084, 0.00001, 11.37822])
        self.assertEqual(trail.result, 1.33084)

        trail.plot()


unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestNewtonRaphson))

# Secante

https://numericalculator.canbula.com/secant

In [ ]:
def secant(f, x0, x1, tolerance=None, max_iterations=None, truncate=True, precision=4):
    cap = trunc if truncate == True else round
    trail = SecantTrail(f, cap, precision)
    trail.record(x0)

    i = 2
    while True:
        trail.record(x1)

        if tolerance is not None and abs(f(x1)) < tolerance:
            break

        if max_iterations is not None and i >= max_iterations:
            break

        result = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        x0, x1 = x1, cap(result, precision)

        i += 1

    return trail

In [ ]:
class TestSecant(unittest.TestCase):
    def test_first_exercise(self):
        trail = secant(
            f=lambda x: x**2 + x - 6,
            x0=1.5,
            x1=1.75,
            tolerance=0.01,
            max_iterations=5,
            precision=5,
        )

        self.assertEqual(trail[0], [1.50000, -2.25000])
        self.assertEqual(trail[1], [1.75000, -1.18750])
        self.assertEqual(trail[2], [2.02941, 0.14791])
        self.assertEqual(trail[3], [1.99846, -0.00769])
        self.assertEqual(trail.result, 1.99846)

        trail.plot()

    def test_second_exercise(self):
        trail = secant(
            f=lambda x: x**3 - 9 * x + 3,
            x0=0,
            x1=1,
            tolerance=5 * 10e-4,
            max_iterations=5,
            truncate=False,
            precision=9,
        )

        self.assertEqual(trail[0], [0.000000000, 3.000000000])
        self.assertEqual(trail[1], [1.000000000, -5.000000000])
        self.assertEqual(trail[2], [0.375000000, -0.322265625])
        self.assertEqual(trail[3], [0.331941545, 0.049101137])
        self.assertEqual(trail[4], [0.337634621, -0.000222209])
        self.assertEqual(trail.result, 0.337634621)

        trail.plot()

    def test_third_exercise(self):
        trail = secant(
            f=lambda x: x**3 - 5 * x**2 + 17 * x + 21,
            x0=-1,
            x1=1,
            max_iterations=7,
            truncate=False,
            precision=10,
        )

        self.assertEqual(trail[0], [-1.0000000000, -2.0000000000])
        self.assertEqual(trail[1], [1.0000000000, 34.0000000000])
        self.assertEqual(trail[2], [-0.8888888889, 1.2359396430])
        self.assertEqual(trail[3], [-0.9601423487, -0.8169162008])
        self.assertEqual(trail[4], [-0.9317876516, 0.0094644431])
        self.assertEqual(trail[5], [-0.9321123947, 0.0000712188])
        self.assertEqual(trail[6], [-0.9321148569, -0.0000000068])

        self.assertEqual(trail.result, -0.9321148569)

        trail.plot()


unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestSecant))